<h2>import data from excel</h2>

In [2]:
import pandas as pd
import numpy as np
import csv
df={}
Demand_num=10 #follow the input file
Demand = list()
Demand = list(range(1,Demand_num+1))
df=pd.read_csv("./prj1_test_data/Car_Demand_"+str(Demand_num)+".csv")
Type_d={}
Location_d={}
Time={}
Penalty={}
for i in range(1,Demand_num+1):
    Location_d[i] = list()
    Time[i] = list()
    Type_d[i]=df.iloc[i-1]["type"]
    Location_d[i].append(df.iloc[i-1]["Location_X"])
    Location_d[i].append(df.iloc[i-1]["Location_Y"])
    Time[i].append(df.iloc[i-1]["Time_H"])
    Time[i].append(df.iloc[i-1]["Time_E"])
    Penalty[i]=df.iloc[i-1]["Penalty"]


In [24]:
print(Demand)
print()
print(Type_d)
print()
print(Location_d)
print()
print(Time)
print()
print(Penalty)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

{1: 3, 2: 3, 3: 2, 4: 1, 5: 2, 6: 3, 7: 1, 8: 1, 9: 2, 10: 3}

{1: [8, 20], 2: [21, 1], 3: [8, 9], 4: [36, 14], 5: [18, 36], 6: [49, 29], 7: [16, 7], 8: [13, 41], 9: [42, 25], 10: [37, 22]}

{1: [748, 886], 2: [173, 545], 3: [296, 655], 4: [61, 524], 5: [824, 981], 6: [644, 678], 7: [222, 625], 8: [713, 932], 9: [909, 927], 10: [310, 749]}

{1: 446, 2: 426, 3: 234, 4: 488, 5: 144, 6: 309, 7: 86, 8: 353, 9: 180, 10: 430}


In [4]:
df2={}
Supply_num=10 
Supply = list()
Supply = list(range(1,Supply_num+1))
df2=pd.read_csv("./prj1_test_data/Space_"+str(Supply_num)+".csv")
Type_s={}
Location_s={}
for i in range(1,Supply_num+1):
    Location_s[i]=list()
    Type_s[i]=df2.iloc[i-1]["type"]
    Location_s[i].append(df2.iloc[i-1]["Location_X"])
    Location_s[i].append(df2.iloc[i-1]["Location_Y"])

In [5]:
print(Supply)
print(Type_s)
print(Location_s)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
{1: 3, 2: 1, 3: 3, 4: 3, 5: 2, 6: 1, 7: 3, 8: 2, 9: 3, 10: 3}
{1: [43, 34], 2: [49, 22], 3: [45, 1], 4: [30, 17], 5: [37, 40], 6: [7, 17], 7: [5, 22], 8: [48, 26], 9: [12, 15], 10: [4, 8]}


In [19]:
def check_time_overlap(a,b):
    if (a[0]<b[0]):
        if (b[0]-a[0]) < (a[1] - a[0]):
            return True
        else:
            return False
    elif (a[0]>b[0]):
        if (a[0]-b[0]) < (b[1] - b[0]):
            return True
        else:
            return False
    else :
        return True
        
        

In [32]:
check_time_overlap([100,150],[0,100])
# print(Time[1])

False

In [23]:
from gurobipy import*
import math

In [43]:
model = Model("Car_parking")
x={}
y={}

<h2>init var</h2>

In [45]:
for d in Demand:
    y[d] = model.addVar(vtype = "B",name = "y(%s)"%d)
    for s in Supply:
        x[d,s] = model.addVar(vtype="B",name = "x(%s,%s)"%(d,s))
model.update()

        

In [39]:
print(Demand)
print()
print(Supply)
print()
print(Type_d)
print()
print(Type_s)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

{1: 3, 2: 3, 3: 2, 4: 1, 5: 2, 6: 3, 7: 1, 8: 1, 9: 2, 10: 3}

{1: 3, 2: 1, 3: 3, 4: 3, 5: 2, 6: 1, 7: 3, 8: 2, 9: 3, 10: 3}


<h1>add constraint</h1>
<h2>time constraint</h2>
<br>
<h3>one demand correspond to one supply</h3>
<h3>But a supply can correspond to several demand</h3>
<p>quicksum(x[d,s] for s in Supply) = 1 for d in Demand</p>
<br>
<h3>Type constraint</h3>
   

In [50]:
R=20
#demand constraint
for d in Demand:
    model.addConstr(quicksum(x[d,s] for s in Supply) == 1,name = "Demand constraint")
    
#time constraint
for i in Demand:
    for j in range(i+1,Demand_num+1):
        if check_time_overlap(Time[i],Time[j]) == True:
            for s in Supply:
                model.addConstr((x[i,s]+x[j,s]) <= 1, name = "Time overlap constraint" ) 
#type constraint
for d in Demand:
    for s in Supply:
        if Type_d[d] > Type_d[s]:
            model.addConstr(x[d,s] == 0, name = "Type constraint")
            
#distance constriant 
r=math.sqrt((Location_d[d][0]-Location_s[s][0])**2+(Location_d[d][1]-Location_s[s][1])**2)
for d in Demand:
    for s in Supply:
        model.addConstr(r*x[d,s]<=R, name = "distance constraint")

In [ ]:
r=math.sqrt((Location_d[d][0]-location_s[s][0])**2+(Location_d[d][1]-location_s[s][1])**2)
model.setObjective(quicksum(Penalty[d]/r)*x[d,s] for (d,s) in x)-(Penalty[d])*y[d])